In [ ]:
!pip install -U scikit-fuzzy
!git clone  https://github.com/cyrus1123/FuzzyCevae.git
%cd FuzzyCevae
!unzip pyro-dev.zip
%cd pyro-dev
%cd pyro-dev
!python setup.py install
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import tensorflow as tf
from __future__ import absolute_import, division, print_function
import sys
import matplotlib.pyplot as plt
import logging

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import pyro
import pyro.distributions as dist
from pyro import poutine
from pyro.infer import SVI, Trace_ELBO
from pyro.infer.util import torch_item
from pyro.nn import PyroModule
from pyro.optim import ClippedAdam
from pyro.util import torch_isnan

logger = logging.getLogger(__name__)


In [8]:
z = ctrl.Antecedent(np.arange(0, 1, 0.01), 'z')
y = ctrl.Antecedent(np.arange(0, 1, 0.01), 'y')
out = ctrl.Consequent(np.arange(0, 1, 0.01), 'out')

# Auto-membership function population is possible with .automf(3, 5, or 7)
y.automf(3)
z.automf(3)

# Custom membership functions can be built interactively with a familiar,
# Pythonic API
out['low'] = fuzz.trimf(out.universe, [0, 0, 0.3])
out['medium'] = fuzz.trimf(out.universe, [0, 0.5, 1])
out['high'] = fuzz.trimf(out.universe, [0.5, 1, 1])
#_____________________________________
rule1 = ctrl.Rule(y['poor'] | z['poor'], out['low'])
rule2 = ctrl.Rule(y['average'], out['medium'])
rule3 = ctrl.Rule(y['good'] | z['good'], out['high'])
#_____________________________________
tipping_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
tipping = ctrl.ControlSystemSimulation(tipping_ctrl)


In [10]:
!pip install causalml
 

     |████████████████████████████████| 245kB 5.4MB/s 
     |████████████████████████████████| 20.1MB 42.6MB/s 
     |████████████████████████████████| 327kB 40.2MB/s 
     |████████████████████████████████| 1.8MB 37.4MB/s 
  Created wheel for causalml: filename=causalml-0.10.0-cp37-cp37m-linux_x86_64.whl size=493679 sha256=26745d5f786413837bc87f5c63dee9f8a6dda8c5fd3e7f1bd8900972d0f9d2e1
  Stored in directory: /root/.cache/pip/wheels/81/7f/44/c9d5ecf03f0d950f53302a5eab1c76bec07bcd5868753e22bf
  Created wheel for shap: filename=shap-0.37.0-cp37-cp37m-linux_x86_64.whl size=465040 sha256=fa0e5424c8460a0b6e757943721d7d091d87a4189803efd3a7a5b9ffb3925c99
  Stored in directory: /root/.cache/pip/wheels/df/ad/b0/aa7815ec68850d66551ef618095eccb962c8f6022f1d3dd989
Successfully built causalml shap
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is i

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import torch

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error as mse
from scipy.stats import entropy
import warnings
import logging

from causalml.inference.meta import BaseXRegressor, BaseRRegressor, BaseSRegressor, BaseTRegressor
from causalml.propensity import ElasticNetPropensityModel
from causalml.metrics import *
from causalml.dataset import simulate_hidden_confounder

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df= pd.read_csv('https://raw.githubusercontent.com/AMLab-Amsterdam/CEVAE/master/datasets/IHDP/csv/ihdp_npci_1.csv', header = None)

df.dataframeName = 'data'

cols =  ["treatment", "y_factual", "y_cfactual", "mu0", "mu1"] + [i for i in range(25)]

df.columns = cols
df.head()

df = pd.concat([df]*100, ignore_index=True)
print(df.shape)

(74700, 30)


In [ ]:
#precising variables type
binfeats = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
contfeats = [i for i in range(25) if i not in binfeats]
perm = binfeats + contfeats
df = df.reset_index(drop=True)
df.head()

In [ ]:
X = df[perm].values
treatment = df['treatment'].values
y = df['y_factual'].values
y_cf = df['y_cfactual'].values
tau = df.apply(lambda y: y['y_factual'] - y['y_cfactual'] if y['treatment']==1 
               else y['y_cfactual'] - y['y_factual'], 
               axis=1)
mu_0 = df['mu0'].values
mu_1 = df['mu1'].values

In [ ]:
# train and test
itr, ite = train_test_split(np.arange(X.shape[0]), test_size=0.2, random_state=1)
X_train, treatment_train, y_train, y_cf_train, tau_train, mu_0_train, mu_1_train = X[itr], treatment[itr], y[itr], y_cf[itr], tau[itr], mu_0[itr], mu_1[itr]
X_val, treatment_val, y_val, y_cf_val, tau_val, mu_0_val, mu_1_val = X[ite], treatment[ite], y[ite], y_cf[ite], tau[ite], mu_0[ite], mu_1[ite]

In [ ]:
# model settings
outcome_dist = "normal"
latent_dim = 20
hidden_dim = 300
num_epochs = 5
batch_size = 1000
learning_rate = 0.001
learning_rate_decay = 0.01
num_layers =20

In [ ]:
import logging
import torch
from pyro.contrib.cevae import CEVAE as CEVAEModel

from causalml.inference.meta.utils import convert_pd_to_np

pyro_logger = logging.getLogger("pyro")
pyro_logger.setLevel(logging.DEBUG)
if pyro_logger.handlers:
    pyro_logger.handlers[0].setLevel(logging.DEBUG)


class CEVAE:
    def __init__(self, outcome_dist="studentt", latent_dim=20, hidden_dim=200, num_epochs=50, num_layers=3,
                 batch_size=100, learning_rate=1e-3, learning_rate_decay=0.1, num_samples=1000, weight_decay=1e-4):
        """
        Initializes CEVAE.
            Args:
                outcome_dist (str): Outcome distribution as one of: "bernoulli" , "exponential", "laplace", "normal",
                                    and "studentt"
                latent_dim (int) : Dimension of the latent variable
                hidden_dim (int) : Dimension of hidden layers of fully connected networks
                num_epochs (int): Number of training epochs
                num_layers (int): Number of hidden layers in fully connected networks
                batch_size (int): Batch size
                learning_rate (int): Learning rate
                learning_rate_decay (float/int): Learning rate decay over all epochs; the per-step decay rate will
                                                 depend on batch size and number of epochs such that the initial
                                                 learning rate will be learning_rate and the
                                                 final learning rate will be learning_rate * learning_rate_decay
                num_samples (int) : Number of samples to calculate ITE
                weight_decay (float) : Weight decay
        """
        self.outcome_dist = outcome_dist
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim
        self.num_epochs = num_epochs
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.learning_rate_decay = learning_rate_decay
        self.num_samples = num_samples
        self.weight_decay = weight_decay

    def fit(self, X, treatment, y, p=None):
        """
        Fits CEVAE.
        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series): a treatment vector
            y (np.array or pd.Series): an outcome vector
        """
        X, treatment, y = convert_pd_to_np(X, treatment, y)

        self.cevae = CEVAEModel(outcome_dist=self.outcome_dist,
                           feature_dim=X.shape[-1],
                           latent_dim=self.latent_dim,
                           hidden_dim=self.hidden_dim,
                           num_layers=self.num_layers)

        self.cevae.fit(x=torch.tensor(X, dtype=torch.float),
                       t=torch.tensor(treatment, dtype=torch.float),
                       y=torch.tensor(y, dtype=torch.float),
                       num_epochs=self.num_epochs,
                       batch_size=self.batch_size,
                       learning_rate=self.learning_rate,
                       learning_rate_decay=self.learning_rate_decay,
                       weight_decay=self.weight_decay)

    def predict(self, X, treatment=None, y=None, p=None):
        """
        Calls predict on fitted DragonNet.
        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
        Returns:
            (np.ndarray): Predictions of treatment effects.
        """
        return self.cevae.ite(torch.tensor(X, dtype=torch.float),
                              num_samples=self.num_samples,
                              batch_size=self.batch_size).cpu().numpy()

    def fit_predict(self, X, treatment, y, p=None):
        """
        Fits the CEVAE model and then predicts.
        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series): a treatment vector
            y (np.array or pd.Series): an outcome vector
        Returns:
            (np.ndarray): Predictions of treatment effects.
        """
        self.fit(X, treatment, y)
        return self.predict(X)

In [ ]:
# from  model import CEVAE

cevae_model = CEVAE(outcome_dist=outcome_dist,
              latent_dim=latent_dim,
              hidden_dim=hidden_dim,
              num_epochs=num_epochs,
              batch_size=batch_size,
              learning_rate=learning_rate,
              learning_rate_decay=learning_rate_decay,
              num_layers=num_layers)

In [ ]:
# fit
losses = cevae_model.fit(X=torch.tensor(X_train, dtype=torch.float),
                   treatment=torch.tensor(treatment_train, dtype=torch.float),
                   y=torch.tensor(y_train, dtype=torch.float))

INFO 	 Training with 60 minibatches per epoch
DEBUG 	 step     0 loss = 78.0852
DEBUG 	 step   100 loss = 24.8715
DEBUG 	 step   200 loss = 22.608


In [ ]:
#train 
ite_train = cevae_model.predict(X_train)
ite_val = cevae_model.predict(X_val)

INFO 	 Evaluating 60 minibatches
DEBUG 	 batch ate = 4.01588
DEBUG 	 batch ate = 4.02003
DEBUG 	 batch ate = 4.02169
DEBUG 	 batch ate = 4.018
DEBUG 	 batch ate = 4.02057
DEBUG 	 batch ate = 4.00867
DEBUG 	 batch ate = 4.02758
DEBUG 	 batch ate = 4.01617
DEBUG 	 batch ate = 4.02131
DEBUG 	 batch ate = 4.02971
DEBUG 	 batch ate = 4.00292
DEBUG 	 batch ate = 3.99255
DEBUG 	 batch ate = 4.01851
DEBUG 	 batch ate = 4.02866
DEBUG 	 batch ate = 4.02144
DEBUG 	 batch ate = 4.00985
DEBUG 	 batch ate = 4.00614
DEBUG 	 batch ate = 4.00623
DEBUG 	 batch ate = 4.04216
DEBUG 	 batch ate = 4.02576
DEBUG 	 batch ate = 4.02717
DEBUG 	 batch ate = 3.99947
DEBUG 	 batch ate = 4.0291
DEBUG 	 batch ate = 4.01242
DEBUG 	 batch ate = 4.02471
DEBUG 	 batch ate = 3.98554
DEBUG 	 batch ate = 4.00191
DEBUG 	 batch ate = 4.03238
DEBUG 	 batch ate = 4.01898
DEBUG 	 batch ate = 4.02727
DEBUG 	 batch ate = 3.99057
DEBUG 	 batch ate = 4.03328
DEBUG 	 batch ate = 4.01537
DEBUG 	 batch ate = 4.01575
DEBUG 	 batch ate 

In [ ]:
ate_train = ite_train.mean()
ate_val = ite_val.mean()
print(ate_train, ate_val)

4.0183487 4.0161624
